# **Tâche #2 - Questions-réponses avec un modèle QA extractif**

Cette tâche consiste à utiliser un modèle de question-réponse extractif de type transformer afin de repérer des informations dans un texte. Vous utilisez la librairie HuggingFace pour accomplir cette tâche. On demande plus spécifiquement d’utiliser le modèle *bert-base-uncased-whole-word-masking-finetuned-squad*.

La tâche a pour but précis de repérer 3 informations dans les descriptions textuelles : le lieu et la date de l’incident ainsi qu’un court passage de texte indiquant ce qui s’est passé.  Une partie importante de votre travail consiste à trouver de bonnes formulations de questions pour repérer ces informations. Le fichier *t2_qa_examples*.json, qui contient 25 exemples annotés par un humain, est disponible pour mener vos expérimentations.

Les consignes pour cette tâche sont:
-	Nom du notebook : *t2_qa.ipynb* (ce notebook)
-	Tokenisation et plongements de mots : Ceux du modèle utilisé.
-	Normalisation : Aucune normalisation à faire (le tokeniseur convertit les lettres en minuscule).
-	Construction du modèle : vous utilisez la version préentraînée du modèle sans modification. Aucun affinement (fine-tuning) du modèle n’est requis pour cette tâche.
-	Évaluation : Du code est disponible dans le notebook pour évaluer la performance du modèle avec les métriques *exact match* et *F1*.
-	Analyse : Présentez et discutez des résultats que vous obtenez pour les 3 types d’informations à repérer. Discutez également de vos choix de questions pour accomplir cette tâche et les erreurs commises par le modèle QA.

Vous pouvez ajouter au notebook toutes les cellules dont vous avez besoin pour votre code, vos explications ou la présentation de vos résultats. Vous pouvez également ajouter des sous-sections (par ex. des sous-sections 1.1, 1.2 etc.) si cela améliore la lisibilité.

Notes :
- Évitez les bouts de code trop longs ou trop complexes. Par exemple, il est difficile de comprendre 4-5 boucles ou conditions imbriquées. Si c'est le cas, définissez des sous-fonctions pour refactoriser et simplifier votre code.
- Expliquez sommairement votre démarche.
- Expliquez les choix que vous faites au niveau de la programmation et des modèles (si non trivial).
- Analysez vos résultats. Indiquez ce que vous observez, si c'est bon ou non, si c'est surprenant, etc.
- Une analyse quantitative et qualitative d'erreurs est intéressante et permet de mieux comprendre le comportement d'un modèle.

## 1. Le chargement des données

Utilisez le fichier ***/data/t2_qa_examples.json*** pour mener vos expérimentations. 

In [23]:
import json

def load_json_data(filename):
    with open(filename, 'r') as fp:
        data = json.load(fp)
    return data

In [24]:
data = load_json_data('data/t2_qa_examples.json')
text = list(map(lambda x: x['text'], data))
expected_when = list(map(lambda x: x['WHEN'], data))
expected_where = list(map(lambda x: x['WHERE'], data))
expected_event = list(map(lambda x: x['EVENT'], data))
print(data)

[{'text': ' At around 10:00 p.m. on November 10  2013  Employee #1  with Villager  Construction Inc.  with a coworker  were using an asphalt milling machine  (Wirtgen; Model Number: W2100) to grind out existing asphalt from an  interstate at a railroad bridge overpass. Employee # 1 was standing on the  ground  checking the depth of the cut into the asphalt  using a handheld  pendant attached to the machine. The pedant could stretch out from ten to 15  ft. This allowed Employee #1 to walk back and forth  checking the cut. The  operator was on the top of the milling machine  controlling the operation of  the machine and ensuring that the milling machine and dump truck (driven by a  second coworker  who worked for an independent trucking service) kept a safe  working distance. A different company  Protective Services Inc. (PSI)  was  responsible for the traffic control of the job site and had shut down the  inside lane of a three lane section of the interstate  so that work could be  cond

## 2. Vos questions 

Vous pouvez mettre plusieurs options de questions dans le notebook. Il est important de présenter, au minimum, les résultats pour le meilleur jeu de questions. Vous pourrez également mettre des informations à ce propos dans la section d'analyse. 

In [167]:
questions_when = [
    "When?",
    "When did the event happen?",
    "What date did the event happen?",
    "What date did the event happen? If no specific date is mentioned, return an empty string.",
]

In [168]:
questions_where = [
    "Where did the event happen?",
    "Where did the incident happen?",
    "Where did it happen?",
    "Name the place where the event happened.",
    "Name the place where the incident happened.",
    "Describe the place where the incident happened.",
    "Find the place where the incident happened.",
    "Identify the place where the incident happened.",
    "Where did the event occured?",
    "Where did the event take place?",
    "Where did the event was located?",
    "Where did the event emerged?",
    "What place did the event occured?",
    "What place did the event occur?",
    "Where?"
]

In [169]:
questions_event = [
    "What?",
    "What is the event?",
    "What is the incident?",
    "What incident has the employee suffered?",
    "What incident has the employee experienced?",
    "What incident has the employee experienced? Mention if it is the employee #1 that has experienced the incident.",
    "Identify the incident that occurred.",
    "Identify and summerize the incident that occurred.",
    "Identify and summerize the incident that the employee experienced.",
    "Identify and summerize the consequences of the incident.",
    "What is the injury that the employee has suffered?",
    "what caused the injury?",
    "What is the cause of the incident?",
    "What is the cause of the injury?",
    "Who was involved in the incident and what happened?",
    "Summerize the incident",
    "Summerize the incident that occurred",
    "Summerize the incident and the consequences",
]

## 3. Le modèle de question-réponse extractif

In [170]:
from transformers import pipeline

model_name = "google-bert/bert-large-uncased-whole-word-masking-finetuned-squad"

model = pipeline(model=model_name, task='question-answering')

Some weights of the model checkpoint at google-bert/bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [171]:
# Rouler la prédiction d'une question pour chaque rapport 
def predict(model, contexts, question):
    return [model(context=c, question=question) for c in contexts]


## 4. Des fonctions utilitaires pour l'évaluation

In [172]:
import string
import re
from collections import Counter

def remove_articles(text):
    return re.sub(r'\b(a|an|the)\b', ' ', text)

def white_space_fix(text):
    return ' '.join(text.split())

def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)

def lower(text):
    return text.lower()

def normalize_answer(s):
    """Mettre en minuscule et retirer la ponctuation, des déterminants and les espaces."""
    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [173]:
def evaluate_f1(ground_truth, prediction):
    """Normalise les 2 textes, trouve ce qu'il y a en commun et estime précision, rappel et F1."""
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if len(ground_truth_tokens) == 0 or len(prediction_tokens) == 0:
        return int(ground_truth_tokens == prediction_tokens)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def evaluate_exact_match(ground_truth, prediction):
    """Vérifie si les 2 textes sont quasi-identiques."""
    return (normalize_answer(prediction) == normalize_answer(ground_truth))

In [174]:
# Rouler la prédiction de la réponse pour chaque rapport et pour chaque questions 
def run(model, contexts, questions, answers):
    for question in questions:
        predictions = predict(model, contexts, question)
        
        # Calcul du F1 score et du exact match
        f1 = 0
        exact_match = 0
        for pred, answer in zip(predictions, answers):
            f1 += evaluate_f1(pred["answer"], answer)
            exact_match += 1 if evaluate_exact_match(pred["answer"], answer) else 0
        print(f"Question: {question}")
        print(f"F1 score: {f1}/{len(contexts)}")
        print(f"Exact match: {exact_match}/{len(contexts)}")
        
        # Lister tous les erreurs de prédictions
        for i, (p, e) in enumerate(zip(predictions, answers)):
            if p['answer'] != e:
                print(f"Prediction: {p} --- Expected: {e}")
        print()

## 5. Évaluation du modèle et analyse

### 5.1 When

In [175]:
run(model, text, questions_when, expected_when)

Question: When?
F1 score: 23.0/25
Exact match: 23/25
Prediction: {'score': 0.2796871066093445, 'start': 155, 'end': 214, 'answer': 'when it bumped up against a ceiling pipe while being raised'} --- Expected: 
Prediction: {'score': 0.31268927454948425, 'start': 57, 'end': 64, 'answer': 'drowned'} --- Expected: 

Question: When did the event happen?
F1 score: 23.0/25
Exact match: 23/25
Prediction: {'score': 0.13847969472408295, 'start': 155, 'end': 214, 'answer': 'when it bumped up against a ceiling pipe while being raised'} --- Expected: 
Prediction: {'score': 0.29438358545303345, 'start': 57, 'end': 64, 'answer': 'drowned'} --- Expected: 

Question: What date did the event happen?
F1 score: 23.0/25
Exact match: 23/25
Prediction: {'score': 0.033587586134672165, 'start': 155, 'end': 214, 'answer': 'when it bumped up against a ceiling pipe while being raised'} --- Expected: 
Prediction: {'score': 0.3679411709308624, 'start': 57, 'end': 64, 'answer': 'drowned'} --- Expected: 

Question: Wh

### 5.2 Where

In [176]:
run(model, text, questions_where, expected_where)

Question: Where did the event happen?
F1 score: 19.919658119658123/25
Exact match: 16/25
Prediction: {'score': 0.10176228731870651, 'start': 193, 'end': 226, 'answer': 'process of mixing cement/concrete'} --- Expected: mixing cement/concrete
Prediction: {'score': 0.0952552855014801, 'start': 193, 'end': 222, 'answer': 'Home Depot  Store Number 6555'} --- Expected: the garden center
Prediction: {'score': 0.06973588466644287, 'start': 279, 'end': 349, 'answer': 'between the excavation wall and the  north wall of the inlet structure'} --- Expected: G-2 drainage inlet structure
Prediction: {'score': 0.023866726085543633, 'start': 42, 'end': 80, 'answer': 'building a wood frame detached  garage'} --- Expected: a wood frame detached  garage
Prediction: {'score': 0.026094749569892883, 'start': 401, 'end': 431, 'answer': 'Employee #1 entered the trench'} --- Expected: in the trench
Prediction: {'score': 0.1536531150341034, 'start': 41, 'end': 72, 'answer': 'a trench  at a depth of 18 feet'} --

### 5.3 Event

In [177]:
run(model, text, questions_event, expected_event)

Question: What?
F1 score: 5.969571072202651/25
Exact match: 3/25
Prediction: {'score': 0.00557408481836319, 'start': 122, 'end': 145, 'answer': 'asphalt milling machine'} --- Expected: Employee #1  was struck and thrown
Prediction: {'score': 0.030539458617568016, 'start': 1193, 'end': 1246, 'answer': 'Employee #1 attempted to load a number  5 700 Panther'} --- Expected: the Panther which weighed over 2 000  pounds fell on his left foot and pinned it
Prediction: {'score': 0.02636348456144333, 'start': 51, 'end': 132, 'answer': 'Employee #1 of Midwest  Masonry  Inc.  was responsible for using the cement mixer'} --- Expected: He was struck by  the cement mixer
Prediction: {'score': 0.020424477756023407, 'start': 24, 'end': 81, 'answer': 'Employee #1  a "flagger" with Iowa Erosion Control   Inc.'} --- Expected: struck Employee #1
Prediction: {'score': 0.06944622099399567, 'start': 80, 'end': 116, 'answer': 'a piece of traffic control equipment'} --- Expected: The passenger vehicle struck t

## Analyse

Pour les trois questions, nous avons procédé de façon itérative, débutant avec les formulations les plus simples.

### Quand?

Pour les questions sur le moment de l'incident, nous avons immédiatement obtenu des résultats satisfaisants avec 23/25 pour l'« exact match ». Les deux erreurs sont dues au fait que la date n'est pas indiquée dans le rapport d'incident. La réponse retournée devrait alors être vide, cependant, le modèle retourne toujours du texte. Nous avons essayé d'ajouter une instruction au modèle pour ne rien retourner si la réponse n'est pas trouvée, cependant, le modèle ne semble pas avoir une capacité avancée de suivi d'instructions. La réponse retournée est toujours la même, peu importe la formulation de la question. Pour ce type de question, le modèle semble donc être robuste à la formulation de la question. Pour obtenir 25/25, il faudrait donc utiliser une méthode externe au modèle, telle que refuser ou accepter les réponses basées sur leur score.

### Où?

La deuxième question, "Où?", s'est avérée plus difficile que la première. Les réponses étant plus complexes, il n'est pas surprenant que les résultats soient inférieurs à la première question qui avait des dates clairement indiquées. Pour cette question, nous avons premièrement testé "Where did the event happen?" et "Where did the incident happen?" pour établir une référence pour les prochaines itérations. Nous avons ensuite testé différentes structures de questions telles que "[Verbe2] the place where the [nom] [verbe1].", "Where did the [nom] [verbe1]?" et "What place did the [nom] [verbe1]?" où [verbe1] est un choix parmi occurred, took place, was located, emerged, happened et [verbe2] est name, describe, find ou identify. Pour [nom], nous avons testé event et incident. La structure "[Verbe2] the place where the [nom] [verbe1]." a donné des résultats significativement inférieurs aux deux autres. Cela est probablement dû au fait que ce n'est pas formulé comme une question, mais plutôt comme une instruction. Les deux autres structures de questions ont peu d'impact sur les résultats. La meilleure question avec les données de test est "What place did the event occur?" avec un F1 score de 20.46/25 et un exact match de 17/25. Par contre, les autres alternatives sont relativement proches; le classement pourrait varier en utilisant un autre groupe de test. Surprenamment, la question contient une erreur, occurred devrait être occur, mais les résultats sont légèrement inférieurs si nous la corrigeons. Nous avons aussi testé simplement "Where?" ce qui donne de meilleurs résultats que le format par instruction, mais inférieurs à une question complète.

La difficulté de la question vient probablement du fait que plusieurs endroits peuvent être donnés dans le rapport; le modèle doit ainsi être en mesure de trouver non seulement ce qui est un endroit, mais aussi l'endroit qui correspond à la question. Les résultats indiquent aussi que le modèle a obtenu certaines réponses partiellement bonnes, l'exact match étant inférieur au F1 score. Cela peut s'expliquer par le fait que le modèle pouvait retourner le bon endroit, mais ne fournissait pas exactement les informations attendues. Par exemple, pour un exemple, le modèle retourne "drainage inlet structure" alors que la réponse attendue était "G-2 drainage inlet structure". Le modèle a trouvé le bon endroit, mais il lui manque une information, ce qui lui donne un bon F1 score, mais pas un exact match.

### Quoi?

Pour la description de l'incident, l'information a été beaucoup plus difficile à bien extraire. Pour trouver les bonnes informations, le modèle doit bien comprendre la question et le rapport. Contrairement à la deuxième question, le choix des mots a eu beaucoup d'impact. Dans un premier temps, nous avons testé "What is the event?" et "What is the incident?". Même s'il y a seulement un mot de différence entre les deux questions, la première a donné le pire résultat avec un F1 score de 2.34/25 et aucun exact match, et la deuxième est la meilleure parmi toutes les questions que nous avons testées avec un F1 score de 14.75/25 et un exact match de 11/25. Nous avons testé d'autres formulations et le format d'instructions donne de très bons résultats. Par exemple, "Identify and summarize the incident that occurred." donne un F1 score de 13.54/25 et un exact match de 11/25, ce qui est très près du meilleur résultat.

### Remarques générales

Basé sur nos expérimentations, les conclusions que nous avons tirées sont que la clarté de l'information recherchée impacte considérablement la capacité du modèle à retrouver l'information. La réponse à la question "Quand?" est claire alors que la réponse au "Quoi?" est plus subtile. Il y a plusieurs façons de formuler l'information et la longueur est très variable. Deuxièmement, les instructions avec la question ont peu d'impact, surtout sur les questions les plus simples telles que le "Quand?". Le modèle est incapable de ne rien retourner; il trouvera toujours une information à retourner même si elle ne correspond pas à l'information recherchée. Finalement, le choix de la structure et des mots peut avoir un grand impact dans certains cas, alors qu'il importe peu dans les autres cas.